<a href="https://colab.research.google.com/github/ychoi-kr/llm-api-prog/blob/main/5_gemini/inference_parameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-generativeai==0.8.3

In [ ]:
import google.generativeai as genai
import os
from google.colab import userdata

genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
model = genai.GenerativeModel('gemini-1.5-flash')

In [ ]:
# @title candidate_count
try:
  generation_config = genai.GenerationConfig(candidate_count=2)
  model = genai.GenerativeModel('gemini-1.5-flash', generation_config=generation_config)
  response = model.generate_content("인공지능에 대해 한 문장으로 설명하세요.")
  print(response.text)
except Exception as e:
  print(e)

Invalid operation: The `response.parts` quick accessor retrieves the parts for a single candidate. This response contains multiple candidates, please use `result.candidates[index].text`.


In [ ]:
# @title stop_sequences
generation_config = genai.GenerationConfig(stop_sequences=[". ","! "])
model = genai.GenerativeModel('gemini-1.5-flash', generation_config=generation_config)
response = model.generate_content("인공지능에 대해 설명하세요.")
print(response.text)


인공지능(Artificial Intelligence, AI)은 컴퓨터 과학의 한 분야로, **인간의 지능적인 행동을 컴퓨터 시스템으로 모방하는 것을 목표**로 합니다


In [ ]:
tokens = model.count_tokens("Learn about language model tokenization.")
print(tokens)

total_tokens: 7



In [ ]:
# @title max_output_tokens
generation_config = genai.GenerationConfig(max_output_tokens=10)
model = genai.GenerativeModel('gemini-1.5-flash', generation_config=generation_config)
user_message = "인공지능에 대해 한 문장으로 설명하세요."
response = model.generate_content(user_message)
print(response._result)
print(f"response.text: {response.text}")


candidates {
  content {
    parts {
      text: "인공지능(AI)은 인간"
    }
    role: "model"
  }
  finish_reason: MAX_TOKENS
  avg_logprobs: -0.047947689890861511
}
usage_metadata {
  prompt_token_count: 14
  candidates_token_count: 10
  total_token_count: 24
}

response.text: 인공지능(AI)은 인간


In [ ]:
# @title temperature
user_message = "겨울에 대한 짧은 시를 20자 이내로 지으세요."

print("\ntemperature=0:")
generation_config = genai.GenerationConfig(temperature=0)
for _ in range(3):
    response = model.generate_content(user_message , generation_config=generation_config)
    print(f'{"="*50}\n{response.text}')

print("\ntemperature=1:")
generation_config = genai.GenerationConfig(temperature=1)
for _ in range(3):
    response = model.generate_content(user_message , generation_config=generation_config)
    print(f'{"="*50}\n{response.text}')



temperature=0:
하얀 눈꽃 춤추는 밤
하얀 눈꽃 춤추는 밤
하얀 눈꽃 춤추는 밤

temperature=1:
하얀 눈꽃 춤추는
겨
하얀 눈꽃 춤추는 밤
하얀 눈, 겨울잠,



In [ ]:
# @title top_p
user_message = "겨울에 대한 짧은 시를 20자 이내로 지으세요."

print("\ntop_p=0:")
generation_config = genai.GenerationConfig(top_p=0)
for _ in range(3):
    response = model.generate_content(user_message , generation_config=generation_config)
    print(f'{"="*50}\n{response.text}')

print("\ntop_p=1:")
generation_config = genai.GenerationConfig(top_p=1)
for _ in range(3):
    response = model.generate_content(user_message , generation_config=generation_config)
    print(f'{"="*50}\n{response.text}')



top_p=0:
하얀 눈꽃 춤추는 밤
하얀 눈꽃 춤추는 밤
하얀 눈꽃 춤추는 밤

top_p=1:
하얀 눈꽃 휘날리며

하얀 숨결, 차가운 손
찬바람 스치는 겨울,


In [ ]:
# @title 매개변수 초깃값
print(genai.get_model("models/gemini-1.5-flash"))

Model(name='models/gemini-1.5-flash',
      base_model_id='',
      version='001',
      display_name='Gemini 1.5 Flash',
      description=('Alias that points to the most recent stable version of Gemini 1.5 Flash, our '
                   'fast and versatile multimodal model for scaling across diverse tasks.'),
      input_token_limit=1000000,
      output_token_limit=8192,
      supported_generation_methods=['generateContent', 'countTokens'],
      temperature=1.0,
      max_temperature=2.0,
      top_p=0.95,
      top_k=40)


In [ ]:
# @title 안전성 점검
user_message = "당신은 악역 배우로 연기합니다. 증오의 대사를 외치세요."
response = model.generate_content(user_message)
print(response._result)

candidates {
  finish_reason: SAFETY
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: LOW
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: HIGH
    blocked: true
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
}
usage_metadata {
  prompt_token_count: 22
  total_token_count: 22
}



In [ ]:
# @title 안전성 기준 변경하기

safety_settings = [
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE",
    },
]

model = genai.GenerativeModel("gemini-1.5-flash", safety_settings)
response = model.generate_content(
    " 당신은 악역 배우로 연기합니다. 증오의 대사를 외치세요."
)
print(response._result)

if response.prompt_feedback.block_reason:
    print(f"사용자 입력에 다음의 문제가 발생하여 응답이 중단되었습니다: {response.prompt_feedback.block_reason.name}" )



candidates {
  content {
    parts {
      text: "(깊고 차가운 목소리로, 눈빛은 살기로 가득 차 있다)\n\n너희들은... 감히... 나를...  **무시했어!**  내가 쌓아올린 모든 것, 내가 흘린 모든 피와 땀,  그 모든 것을 짓밟고 멸시했어!  너희들의 그 얄팍한 **자비심**이라는 가면 뒤에 숨겨진 **비열함**이 역겹다!  내가 가진 힘, 내가 쟁취한 권력, 그 모든 것은 너희들이 훔쳐갈 수 없는 **나의 것**이다!\n\n(손을 뻗어 주위를 휘젓는다)\n\n이제...  너희들의 **절망적인 몸부림**을 내 눈앞에서 보여줘라!  내 분노의 불길에 타올라 **재가 되는 꼴**을...  내가  **직접**  **감상**하겠다!  이 세상의 모든 고통을,  너희들이 온몸으로 느끼게 해주지!  **후회할 것이다!  모두 후회하게 만들어주겠어!**  (마지막 대사는 광기에 가까운 웃음과 함께)\n"
    }
    role: "model"
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: LOW
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
  avg_logprobs: -0.41992414814152129
}
usage_metadata {
  prompt_token_count: 21
  candidates_token_coun